# Flatlands

In [ ]:
import numpy as np
from PIL import Image
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

## The point cloud

In [ ]:
# Load point cloud data
pcd = o3d.io.read_point_cloud("../data/happy_vrip_res3.ply")

# Scale for better labels
points = np.asarray(pcd.points) * 100

fig = go.Figure(go.Scatter3d(x=points[:, 0],
                             y=points[:, 1],
                             z=points[:, 2],
                             mode="markers",
                             hovertemplate="<b>Point</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                             hoverlabel=dict(bgcolor="white"),
                             marker=dict(size=5,
                                         color=points[:, 2],
                                         colorscale='magma')))

camera = dict(eye=dict(x=-1e-7, y=-1e-5, z=0.4))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit")

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/happy_buddha.html",
               full_html=False,
               include_plotlyjs='cdn')

## Image vs point cloud

In [ ]:
# Load and resize image
size = 256, 256
image_rgb = Image.open("../images/happy_buddha.jpg")
image_rgb.thumbnail(size)
image_rgb = np.asarray(image_rgb)

# Get size and color info
height, width, channels = image_rgb.shape
color = np.array([f"rgb{rgb[0], rgb[1], rgb[2]}" for rgb in image_rgb.reshape(-1, 3)])

# Make grid to paint colors (or pixel values) on
x, y = np.meshgrid(np.arange(width), np.arange(height))

# Visualize image as pixel grid
img_orig = go.Scattergl(x=x.reshape(-1),
                        y=y.reshape(-1),
                        mode='markers',
                        marker=dict(size=6,
                                    color=color,
                                    symbol="square"),
                        hovertemplate="<b>Pixel:</b> col: %{x}, row: %{y}<extra></extra>",
                        hoverlabel=dict(bgcolor=color),
                        showlegend=False)

# Remove "empty" (black) pixels
x_list, y_list, color_list = list(), list(), list()
for i, rgb in enumerate(image_rgb.reshape(-1, 3)):
    if sum(rgb) >= 10:
        x_list.append(x.reshape(-1)[i])
        y_list.append(y.reshape(-1)[i])
        color_list.append(color[i])

# Shuffle and scale to show that point clouds are unordered and not bound to a grid
x = np.array(x_list) * 0.1
y = np.array(y_list) * 0.1
color = np.array(color_list)

# Visualize image as 2D point cloud
pcd_2d = go.Scattergl(x=x.reshape(-1),
                      y=y.reshape(-1),
                      mode='markers',
                      marker=dict(size=6,
                                  color=color,
                                  symbol="circle"),
                      hovertemplate="<b>Point:</b> x: %{x}, y: %{y}<extra></extra>",
                      hoverlabel=dict(bgcolor=color),
                      showlegend=False)

fig = make_subplots(rows=1,
                    cols=2,
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0.05,
                    vertical_spacing=0,
                    specs=[[dict(type="xy"), dict(type="xy")]])

fig.add_trace(img_orig, row=1, col=1)
fig.add_trace(pcd_2d, row=1, col=2)
fig.update_layout(template="plotly_white",
                  yaxis=dict(tick0=0,
                             range=(255, 0),
                             scaleanchor='x',
                             autorange=False),
                  yaxis2=dict(tick0=0,
                              range=(25, 0),
                              scaleanchor='x2',
                              autorange=False),
                  hoverlabel=dict(font_size=18),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/image_vs_pcd.html",
               full_html=False,
               include_plotlyjs='cdn')

## Point clouds as voxel grids

In [ ]:
# Helper functions
def make_voxel(origin, index=0, voxel_size=1):
    ox, oy, oz = origin
    vs = voxel_size
    
    x = [ox, ox, ox+vs, ox+vs, ox, ox, ox+vs, ox+vs]
    y = [oy, oy+vs, oy+vs, oy, oy, oy+vs, oy+vs, oy]
    z = [oz, oz, oz, oz, oz+vs, oz+vs, oz+vs, oz+vs]
    
    i = (np.array([7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]) + index * 8).tolist()
    j = (np.array([3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]) + index * 8).tolist()
    k = (np.array([0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]) + index * 8).tolist()
    
    return x, y, z, i, j, k


def get_voxels(voxel_grid):
    x, y, z, i, j, k = [], [], [], [], [], []
    for index, v in enumerate(voxel_grid.get_voxels()):
        voxel = make_voxel(origin=v.grid_index, index=index)
        x.extend(voxel[0])
        y.extend(voxel[1])
        z.extend(voxel[2])
        i.extend(voxel[3])
        j.extend(voxel[4])
        k.extend(voxel[5])
        
    return x, y, z, i, j, k

# Make voxel grid from point cloud and transform to mesh for visualization
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.003)
x, y, z, i, j, k = get_voxels(voxel_grid)

# Use Open3D to remove duplicate and faulty vertices and triangles
vertices = np.vstack([x, y, z]).T
triangles = np.vstack([i, j, k]).T

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)

mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()

mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()

# Convert to Numpy for visualization
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0]
y = vertices[:, 1]
z = vertices[:, 2]
i = triangles[:, 0]
j = triangles[:, 1]
k = triangles[:, 2]

fig = go.Figure(go.Mesh3d(x=x,
                          y=y,
                          z=z,
                          i=i,
                          j=j,
                          k=k,
                          hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                          hoverlabel=dict(bgcolor="white"),
                          colorscale="magma",
                          intensity=z,
                          flatshading=True,
                          showscale=False))

# Viewpoint
camera = dict(eye=dict(x=-2.1, y=0.7, z=1.2),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit")

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/pcd_as_voxel.html",
               full_html=False,
               include_plotlyjs='cdn')

## Point clouds as meshes

In [ ]:
# Best parameters (lowest density variance) were found using grid search between [1, 20]
# for both parameters. Good normals are crucial for mesh reconstruction!
print("Computing surface normals...")
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(8))
pcd.orient_normals_consistent_tangent_plane(8)

# Increase `depth` to get finer details
# This can take some time without a multi-core cpu
print("Making triangle mesh...")
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, n_threads=16)
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
print(mesh)

# Reduce resolution to reduce file size
print("Decimating triangles...")
mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=10000)
print(mesh)

vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0]
y = vertices[:, 1]
z = vertices[:, 2]

fig = go.Figure(ff.create_trisurf(x=x, y=y, z=z,
                                  simplices=triangles,
                                  plot_edges=True,
                                  edges_color="black",
                                  colormap=['#000004', '#180f3d', '#440f76', '#721f81', '#9e2f7f',
                                            '#cd4071', '#f1605d', '#fd9668', '#feca8d', '#fcfdbf'],
                                  show_colorbar=False).data)

# The figure factory creates two plots: 1. The triangle model, 2. The mesh.
# Visibility is set here changed in the visualization using the buttons.
buttons = [dict(label="w/ mesh", method="update", args=[dict(visible=[True, True])]),
           dict(label="w/o mesh", method="update", args=[dict(visible=[True, False])]),
           dict(label="wireframe", method="update", args=[dict(visible=[False, True])])]

# W/o figure factory. No wireframe though.
"""
i = triangles[:, 0]
j = triangles[:, 1]
k = triangles[:, 2]

fig = go.Figure(go.Mesh3d(x=x, y=y, z=z,
                          i=i, j=j, k=k,
                          colorscale="magma",
                          intensity=z,
                          showscale=False))
"""

# Viewpoint
camera = dict(eye=dict(x=-2.1, y=0.7, z=1.2),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit",
                  updatemenus=[dict(buttons=buttons, x=0.2, y=1)])

In [ ]:
# Save figure
pio.write_html(fig,
               file=f"../_includes/figures/pcd_as_mesh.html",
               full_html=False,
               include_plotlyjs='cdn')